In [0]:
import pandas as pd
import re
import numpy as np
from pyspark.sql.functions import *  # Import all functions from pyspark.sql.functions

In [0]:
# Load the CSV file into a Spark DataFrame
#df = spark.read.format('csv').options(  # Specify the format as CSV and set options
#   header='true',  # First line of the file is a header
#    inferSchema='true',  # Infer the schema of the CSV file
#    multiline='true',  # Allow multiline fields
#    escape="\""  # Escape character for quotes
#).load('/Volumes/edav_dev_ddnid_niosh/wfsp/datafiles/comments_rev1_pandas.csv')  # Load the CSV file from the specified path

df = pd.read_csv('/Volumes/edav_dev_ddnid_niosh/wfsp/datafiles/comments_rev1_pandas.csv')

display(df)

# Display the DataFrame
#display(df)  # Use display() to show the DataFrame

Unnamed: 0,Document_ID,Agency_ID,Docket_ID,Tracking_Number,Document_Type,Posted_Date,Is_Withdrawn?,Federal_Register_Number,FR_Citation,Title,Comment_Start_Date,Comment_Due_Date,Allow_Late_Comments,Comment_on_Document_ID,Effective_Date,Implementation_Date,Postmark_Date,Received_Date,Author_Date,Related_RIN(s),Authors,CFR,Abstract,Legacy_ID,Media,Document_Subtype,Exhibit_Location,Exhibit_Type,Additional_Field_1,Additional_Field_2,Topics,Duplicate_Comments,OMB/PRA_Approval_Number,Page_Count,Page_Length,Paper_Width,Special_Instructions,Source_Citation,Start_End_Page,Subject,First_Name,Last_Name,City,State/Province,Zip/Postal_Code,Country,Organization_Name,Submitter_Representative,Representative's_Address,"Representative's_City,_State_&_Zip",Government_Agency,Government_Agency_Type,Comment,Category,Restrict_Reason_Type,Restrict_Reason,Reason_Withdrawn,Content_Files,Attachment_Files,"Display_Properties_(Name,_Label,_Tooltip)",cluster,Probability,ID,Is_Duplicated,FormattedDate,AfterAug25
0,CDC-2023-0051-0458,CDC,CDC-2023-0051,lkt-9vyi-tgi0,Public Submission,2023-08-16T04:00Z,false,null,null,"Comment from Unites, Caitlin",null,null,false,CDC-2023-0051-0001,null,null,null,2023-08-02T04:00Z,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,Caitlin,Unites,null,null,null,null,null,null,null,null,null,null,"The proposed rule places unnecessary burden on dog owners living overseas. As a public health professional, I understand the importance of preventing the reintroduction of rabies to America, but as a US Diplomat, I am dismayed that CDC has established rules that are excessively onerous. As a USAID Foreign Service Officer, most, if not all, countries that I may be assigned are on CDC's list of high risk countries, and do not have Approved Rabies Serology Laboratories. As a single woman, my dogs provide both companionship and security when living overseas in difficult places. While I will not bid on positions in countries where pets are not permitted, I am deeply concerned about these rules in the event of an unanticipated emergency leading to ordered departure from a post. This could be for a number of reasons from political instability, to an environmental emergency, with very little warning. In the event of an ordered departure, there would not be time (or possibly even in country systems) to obtain a serology test from abroad and get CDC importation approval (the recommended time according to CDC is 8 weeks). Abandoning my dogs or putting my dogs in a 28 day quarantine would be an additional burden in an already difficult time. As a responsible dog owner, I ensure that my dog is up to date on all vaccines. While I understand that these rules are intended to prevent falsified documentation, I urge you to consider an exception for USG employees and service members serving abroad, or at a minimum exceptions for people serving at posts on ordered departure. I make sacrifices to serve my country in difficult posts overseas, and I accept a certain level of risk to do so, but I also expect my country to take care of my family unit, which includes my dogs",null,null,null,null,null,null,"pageCount, Page Count, Number of pages In the content file",1.0,0.5222590115088209,CDC-2023-0051-0458,false,2023-08-16,Before
1,CDC-2023-0051-0459,CDC,CDC-2023-0051,lkt-9yky-dh5i,Public Submission,2023-08-16T04:00Z,false,null,null,"Comment from T, Nathan",null,null,false,CDC-2023-0051-0001,null,null,null,2023-08-02T04:00Z,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,Nathan,T,null,null,null,null,null,null,null,null,null,null,"I am a Foreign Service Officer in the Department of State, proudly serving our country in difficult overseas assignments where our country needs us, since 2005. When our family serves overseas and returns to America, our pets are a source of comfort and support. The proposed rule would dramatically impact our ability to br

In [0]:

df['cluster'] = pd.to_numeric(df['cluster'], errors='coerce').astype('Int64') #convert float into integer
df.head(2)

,Unnamed: 0,Document_ID,Agency_ID,Docket_ID,Tracking_Number,Document_Type,Posted_Date,Is_Withdrawn?,Federal_Register_Number,FR_Citation,Title,Comment_Start_Date,Comment_Due_Date,Allow_Late_Comments,Comment_on_Document_ID,Effective_Date,Implementation_Date,Postmark_Date,Received_Date,Author_Date,Related_RIN(s),Authors,CFR,Abstract,Legacy_ID,Media,Document_Subtype,Exhibit_Location,Exhibit_Type,Additional_Field_1,Additional_Field_2,Topics,Duplicate_Comments,OMB/PRA_Approval_Number,Page_Count,Page_Length,Paper_Width,Special_Instructions,Source_Citation,Start_End_Page,Subject,First_Name,Last_Name,City,State/Province,Zip/Postal_Code,Country,Organization_Name,Submitter_Representative,Representative's_Address,"Representative's_City,_State_&_Zip",Government_Agency,Government_Agency_Type,Comment,Category,Restrict_Reason_Type,Restrict_Reason,Reason_Withdrawn,Content_Files,Attachment_Files,"Display_Properties_(Name,_Label,_Tooltip)",cluster,Probability,ID,Is_Duplicated,FormattedDate,AfterAug25
0,0,CDC-2023-0051-0458,CDC,CDC-2023-0051,lkt-9vyi-tgi0,Public Submission,2023-08-16T04:00Z,False,NaN,NaN,"Comment from Unites, Caitlin",NaN,NaN,False,CDC-2023-0051-0001,NaN,NaN,NaN,2023-08-02T04:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,Caitlin,Unites,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The proposed rule places unnecessary burden on...,NaN,NaN,NaN,NaN,NaN,NaN,"pageCount, Page Count, Number of pages In the ...",1,0.522259,CDC-2023-0051-0458,False,2023-08-16,Before
1,1,CDC-2023-0051-0459,CDC,CDC-2023-0051,lkt-9yky-dh5i,Public Submission,2023-08-16T04:00Z,False,NaN,NaN,"Comment from T, Nathan",NaN,NaN,False,CDC-2023-0051-0001,NaN,NaN,NaN,2023-08-02T04:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,Nathan,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I am a Foreign Service Officer in the Departme...,NaN,NaN,NaN,NaN,NaN,NaN,"pageCount, Page Count, Number of pages In the ...",1,0.330687,CDC-2023-0051-0459,False,2023-08-16,Before


In [0]:
def clean_comment(text):
    if pd.notna(text):  # Check if the text is not NaN
        text = text.lower()  # Convert text to lowercase
        text = text.replace('\\n', ' ')  # Replace escaped newline characters with a space
        text = text.replace('\n', ' ')  # Replace newline characters with a space
        text = re.sub(r'http\S+', '', text)  # Remove URLs
        text = text.encode('ascii',errors='ignore').decode()  # Remove non-ascii characters
        text = re.sub("^\s+|\s+$", "", text, flags=re.UNICODE)  # Remove leading and trailing whitespace
        text = " ".join(re.split("\s+", text, flags=re.UNICODE))  # Replace multiple spaces with a single space
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = re.sub(r'\d', '', text)  # Remove numbers
    else:
        pass  # Do nothing if text is NaN
    return text  # Return the cleaned text

In [0]:
display(df)

Unnamed: 0,Document_ID,Agency_ID,Docket_ID,Tracking_Number,Document_Type,Posted_Date,Is_Withdrawn?,Federal_Register_Number,FR_Citation,Title,Comment_Start_Date,Comment_Due_Date,Allow_Late_Comments,Comment_on_Document_ID,Effective_Date,Implementation_Date,Postmark_Date,Received_Date,Author_Date,Related_RIN(s),Authors,CFR,Abstract,Legacy_ID,Media,Document_Subtype,Exhibit_Location,Exhibit_Type,Additional_Field_1,Additional_Field_2,Topics,Duplicate_Comments,OMB/PRA_Approval_Number,Page_Count,Page_Length,Paper_Width,Special_Instructions,Source_Citation,Start_End_Page,Subject,First_Name,Last_Name,City,State/Province,Zip/Postal_Code,Country,Organization_Name,Submitter_Representative,Representative's_Address,"Representative's_City,_State_&_Zip",Government_Agency,Government_Agency_Type,Comment,Category,Restrict_Reason_Type,Restrict_Reason,Reason_Withdrawn,Content_Files,Attachment_Files,"Display_Properties_(Name,_Label,_Tooltip)",cluster,Probability,ID,Is_Duplicated,FormattedDate,AfterAug25
0,CDC-2023-0051-0458,CDC,CDC-2023-0051,lkt-9vyi-tgi0,Public Submission,2023-08-16T04:00Z,false,null,null,"Comment from Unites, Caitlin",null,null,false,CDC-2023-0051-0001,null,null,null,2023-08-02T04:00Z,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,Caitlin,Unites,null,null,null,null,null,null,null,null,null,null,"The proposed rule places unnecessary burden on dog owners living overseas. As a public health professional, I understand the importance of preventing the reintroduction of rabies to America, but as a US Diplomat, I am dismayed that CDC has established rules that are excessively onerous. As a USAID Foreign Service Officer, most, if not all, countries that I may be assigned are on CDC's list of high risk countries, and do not have Approved Rabies Serology Laboratories. As a single woman, my dogs provide both companionship and security when living overseas in difficult places. While I will not bid on positions in countries where pets are not permitted, I am deeply concerned about these rules in the event of an unanticipated emergency leading to ordered departure from a post. This could be for a number of reasons from political instability, to an environmental emergency, with very little warning. In the event of an ordered departure, there would not be time (or possibly even in country systems) to obtain a serology test from abroad and get CDC importation approval (the recommended time according to CDC is 8 weeks). Abandoning my dogs or putting my dogs in a 28 day quarantine would be an additional burden in an already difficult time. As a responsible dog owner, I ensure that my dog is up to date on all vaccines. While I understand that these rules are intended to prevent falsified documentation, I urge you to consider an exception for USG employees and service members serving abroad, or at a minimum exceptions for people serving at posts on ordered departure. I make sacrifices to serve my country in difficult posts overseas, and I accept a certain level of risk to do so, but I also expect my country to take care of my family unit, which includes my dogs",null,null,null,null,null,null,"pageCount, Page Count, Number of pages In the content file",1,0.5222590115088209,CDC-2023-0051-0458,false,2023-08-16,Before
1,CDC-2023-0051-0459,CDC,CDC-2023-0051,lkt-9yky-dh5i,Public Submission,2023-08-16T04:00Z,false,null,null,"Comment from T, Nathan",null,null,false,CDC-2023-0051-0001,null,null,null,2023-08-02T04:00Z,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,Nathan,T,null,null,null,null,null,null,null,null,null,null,"I am a Foreign Service Officer in the Department of State, proudly serving our country in difficult overseas assignments where our country needs us, since 2005. When our family serves overseas and returns to America, our pets are a source of comfort and support. The proposed rule would dramatically impact our ability to brin

In [0]:
type(df['Comment'])  # Check the type of the 'Comment' column in the DataFrame
# pyspark.sql.column.Column  # The type of the 'Comment' column is a PySpark Column

# clean text for keyword extraction
df['Clean_comment'] = df['Comment'].apply(clean_comment)
df.head(5)

,Unnamed: 0,Document_ID,Agency_ID,Docket_ID,Tracking_Number,Document_Type,Posted_Date,Is_Withdrawn?,Federal_Register_Number,FR_Citation,Title,Comment_Start_Date,Comment_Due_Date,Allow_Late_Comments,Comment_on_Document_ID,Effective_Date,Implementation_Date,Postmark_Date,Received_Date,Author_Date,Related_RIN(s),Authors,CFR,Abstract,Legacy_ID,Media,Document_Subtype,Exhibit_Location,Exhibit_Type,Additional_Field_1,Additional_Field_2,Topics,Duplicate_Comments,OMB/PRA_Approval_Number,Page_Count,Page_Length,Paper_Width,Special_Instructions,Source_Citation,Start_End_Page,Subject,First_Name,Last_Name,City,State/Province,Zip/Postal_Code,Country,Organization_Name,Submitter_Representative,Representative's_Address,"Representative's_City,_State_&_Zip",Government_Agency,Government_Agency_Type,Comment,Category,Restrict_Reason_Type,Restrict_Reason,Reason_Withdrawn,Content_Files,Attachment_Files,"Display_Properties_(Name,_Label,_Tooltip)",cluster,Probability,ID,Is_Duplicated,FormattedDate,AfterAug25,Clean_comment
0,0,CDC-2023-0051-0458,CDC,CDC-2023-0051,lkt-9vyi-tgi0,Public Submission,2023-08-16T04:00Z,False,NaN,NaN,"Comment from Unites, Caitlin",NaN,NaN,False,CDC-2023-0051-0001,NaN,NaN,NaN,2023-08-02T04:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,Caitlin,Unites,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The proposed rule places unnecessary burden on...,NaN,NaN,NaN,NaN,NaN,NaN,"pageCount, Page Count, Number of pages In the ...",1,0.522259,CDC-2023-0051-0458,False,2023-08-16,Before,the proposed rule places unnecessary burden on...
1,1,CDC-2023-0051-0459,CDC,CDC-2023-0051,lkt-9yky-dh5i,Public Submission,2023-08-16T04:00Z,False,NaN,NaN,"Comment from T, Nathan",NaN,NaN,False,CDC-2023-0051-0001,NaN,NaN,NaN,2023-08-02T04:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,Nathan,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I am a Foreign Service Officer in the Departme...,NaN,NaN,NaN,NaN,NaN,NaN,"pageCount, Page Count, Number of pages In the ...",1,0.330687,CDC-2023-0051-0459,False,2023-08-16,Before,i am a foreign service officer in the departme...
2,2,CDC-2023-0051-0460,CDC,CDC-2023-0051,llc-lpp1-7o91,Public Submission,2023-08-16T04:00Z,False,NaN,NaN,Comment from Anonymous,NaN,NaN,False,CDC-2023-0051-0001,NaN,NaN,NaN,2023-08-15T04:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,Anonymous,Anonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I am writing to oppose the proposed changes to...,NaN,NaN,NaN,NaN,NaN,NaN,"pageCount, Page Count, Number of pages In the ...",3,1.000000,CDC-2023-0051-0460,True,2023-08-16,Before,i am writing to oppose the proposed changes to...
3,3,CDC-2023-0051-0461,CDC,CDC-2023-0051,llc-mnxn-8gn8,Public Submission,2023-08-16T04:00Z,False,NaN,NaN,"Comment from Rondeau, Alice",NaN,NaN,False,CDC-2023-0051-0001,NaN,NaN,NaN,2023-08-15T04:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,Alice,Rondeau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n\nI am writing to OPPOSE the proposed change...,NaN,NaN,NaN,NaN,NaN,NaN,"pageCount, Page Count, Number of pages In the ...",3,1.000000,CDC-2023-0051-0461,False,2023-08-16,Before,i am writing to oppose the proposed changes to...
4,4,CDC-2023-0051-0462,CDC,CDC-2023-0051,llc-o46p-147p,Public Submission,2023-08-16T04:00Z,False,NaN,NaN,"Comment from Ahearn, Megan",NaN,NaN,False,CDC-2023-0051-0001,NaN,NaN,NaN,2023-08-15T04:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,Megan,Ahearn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This proposed rule would further punish those ...,NaN,NaN,NaN,NaN,NaN,NaN,"pageCount, Page Count, Number of pages In the ...",1,0.424414,CDC-2023-0051-0462,False,2023-08-16,Before,this proposed rule would further punish those ...


In [0]:
# Keywords for category 1
keywords1 = [
    'altenburger',
    'devastating',
    'safeguards'
]

# Keywords for category 2
keywords2 = [
    'foreign service',
    'foreign service officer',
    'foreign service officer',
    'foreign service employee',
    'foreign service employees',
    'american foreign service association',
    'afsa'
]

In [0]:
# Custom function to return identified keywords in a separate column
def contains_keywords(text, keywords):
    if pd.notna(text):  # Check if the text is not NaN
        found_keywords = []  # Initialize an empty list to store found keywords
        for keyword in keywords:  # Iterate over each keyword in the list
            if keyword in text:  # Check if the keyword is in the text
                found_keywords.append(keyword)  # Append the keyword to the list if found
        return found_keywords  # Return the list of found keywords
    else:
        pass  # Do nothing if the text is NaN

In [0]:
# Apply the custom function 'contains_keywords' to the 'Comment' column of the DataFrame 'df'
# The function checks for the presence of keywords from the 'keywords1' list in each comment
# The result is stored in a new column 'OtherW'
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

# Define the UDF
contains_keywords_udf = udf(lambda x: contains_keywords(x, keywords1), BooleanType())

# Apply the UDF to the 'Comment' column
df = df.withColumn('OtherW', contains_keywords_udf(df['Comment']))

# Display the DataFrame
display(df)

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
~/.ipykernel/33210/command-1011475897609923-1558532164 in ?()
      7 # Define the UDF
      8 contains_keywords_udf = udf(lambda x: contains_keywords(x, keywords1), BooleanType())
      9 
     10 # Apply the UDF to the 'Comment' column
---> 11 df = df.withColumn('OtherW', contains_keywords_udf(df['Comment']))
     12 
     13 # Display the DataFrame
     14 display(df)

/databricks/python/lib/python3.10/site-packages/pandas/core/generic.py in ?(self, name)
   5898             and name not in self._accessors
   5899             and self._info_axis._can_hold_identifiers_and_holds_name(name)
   5900         ):
   5901             return self[name]
-> 5902         return object.__getattribute__(self, name)

AttributeError: 'DataFrame' object has no attribute 'withColumn'

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

# Define the UDF (User Defined Function) to check for keywords in the 'Comment' column
contains_keywords_udf = udf(lambda x: contains_keywords(x, keywords2), BooleanType())

# Apply the UDF to create a new column 'ForServ' that indicates the presence of keywords from 'keywords2' in the 'Comment' column
df = df.withColumn('ForServ', contains_keywords_udf(df['Comment']))

# Display the DataFrame with the new 'ForServ' column
display(df)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
from pyspark.sql.functions import when

# Recode cluster as 'duplicated_or_empty' based on True values in Is_Duplicated column
df = df.withColumn('cluster', when(df['Is_Duplicated'], 'duplicated_or_empty').otherwise(df['cluster']))

df.tail(6)  # Display the last 6 rows of the DataFrame

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
# create group of duplicates within all duplicated comments
df_duplicated = df[df['cluster'] == 'duplicated_or_empty']  # Filter the DataFrame to include only rows where the 'cluster' column is 'duplicated_or_empty'
df_duplicated.shape  # Get the shape (number of rows and columns) of the filtered DataFrame

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
# Select 'Document_ID' and 'Clean_comment' columns from the 'df_duplicated' DataFrame
df_dupl_selCols = df_duplicated.loc[:, ['Document_ID','Clean_comment']]

# Display the first 2 rows of the 'df_dupl_selCols' DataFrame
df_dupl_selCols.head(2)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
# Create a dictionary 'text_to_group' where each unique 'Clean_comment' text is mapped to a unique group number
text_to_group = {text: group_num for group_num, text in enumerate(df_dupl_selCols['Clean_comment'].unique())}

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
# Map each 'Clean_comment' to its corresponding group number using 'text_to_group' dictionary and create a new column 'DuplGr'
df_dupl_selCols['DuplGr'] = df_dupl_selCols['Clean_comment'].map(text_to_group)

# Display the first 6 rows of the updated 'df_dupl_selCols' DataFrame
df_dupl_selCols.head(6)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
# Get the counts of each unique group number in the 'DuplGr' column
df_dupl_selCols['DuplGr'].value_counts()

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
#keep only ID and dupl group for merging
df_dupl_selCols.drop('Clean_comment', axis = 1, inplace = True)
df_dupl_selCols.head(2)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
#left join 
df_merged = pd.merge(df, df_dupl_selCols, how='left', left_on=['Document_ID'],right_on=['Document_ID'])  # Perform a left join between df and df_dupl_selCols on 'Document_ID'
df_merged['DuplGr'] = pd.to_numeric(df_merged['DuplGr'], errors='coerce').astype('Int64')  # Convert 'DuplGr' column to numeric, coercing errors, and then to integer type

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
# Print the shape of the merged DataFrame
print(df_merged.shape)

# Display the first 4 rows of the merged DataFrame
df_merged.head(4)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
#create cluster_labelled column
mapping = {
0: '0_partSupport_support',
1: '1_againstRule_homeless',
2: '2_govWorkers_varDocs',
3: '3_breeders',
4: '4_govWorkers_burden',
5: '5_rescue',
6: '6_saveDogs_short',
7: '7_govWorkers_short',
8: '8_fairTreat_animals',
9: '9_rescue_cdc_opinions',
10: '10_doNot_kill_short',
11: '11_animal_experiment',
12: '12_petOwners_except',
13: '13_doNot_kill_redTape',
14: '14_otherArguments',
'duplicated_or_empty':'duplicated_or_empty'
}

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
# Map the 'cluster' column to the 'mapping' dictionary and create a new column 'cluster_labelled'
df_merged['cluster_labelled'] = df_merged['cluster'].map(mapping)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
# Rename the 'Probability' column to 'Prob' to save space in the dashboard
df_merged.rename(columns = {'Probability': 'Prob'}, inplace = True)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
df_merged.head(2)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable

In [0]:
# Save the DataFrame 'df_merged' to a CSV file named "15_cl_duplpairs.csv"
df_merged.to_csv("15_cl_duplpairs.csv")

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3123365896563181>, line 5
      1 #type(df['Comment'])
      2 #pyspark.sql.column.Column
      3 
      4 # clean text for keyword extraction
----> 5 df['Clean_comment'] = df['Comment'].apply(clean_comment)
      6 df.head(5)

TypeError: 'Column' object is not callable